<a href="https://colab.research.google.com/github/iubordei/Ahorcado/blob/master/practica9_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Práctica 9 Parte 3: Desarrollando un modelo de lenguaje para generar texto

Un modelo de lenguaje puede predecir la siguiente palabra de una secuencia basándose en palabras observadas anteriormente. Las redes neuronales son el método más utilizado para desarrollar este tipo de modelos porque pueden usar una representación donde palabras con significados similares tienen representaciones similares. 

En esta parte de la práctica vamos a ver cómo generar uno de esos modelos. 

Este notebook está basado en el libro Deep Learning for Natural Language Processing de Jason Brownlee. 

Es importante que tengas activado el uso de **GPU** en el notebook de colab (menú Edit -> Notebook Settings -> Hardware accelerator).

## La República de Platón

Nuestro modelo de lenguaje va a estar basado en la república de Platón. Este libro está estructurado en forma de una conversación que trata el tema del orden y la justicia dentro de una ciudad. El texto completo está disponible para el dominio público dentro del [proyecto Gutenberg](http://www.gutenberg.org/).

Este libro de Platón está disponible en varios formatos en el [proyecto Gutenberg](http://www.gutenberg.org/cache/epub/1497/pg1497.txt). La versión que nos interesa a nosotros es la versión ASCII del libro. Con la siguiente instrucción puedes descargar el libro donde se han eliminado la portada y la contraportada. 

In [1]:
!wget https://raw.githubusercontent.com/ts1819/datasets/master/practica5/republic.txt -O republic.txt

--2021-05-15 08:33:08--  https://raw.githubusercontent.com/ts1819/datasets/master/practica5/republic.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 657826 (642K) [text/plain]
Saving to: ‘republic.txt’

republic.txt        100%[===================>] 642.41K  --.-KB/s    in 0.04s   

2021-05-15 08:33:08 (17.5 MB/s) - ‘republic.txt’ saved [657826/657826]



## Preparación de los datos

Vamos a preparar los datos para construir nuestro modelo. 

### Revisando el texto

Vamos a comenzar revisando parte del texto.

In [2]:
!head -30 republic.txt

BOOK I.

I went down yesterday to the Piraeus with Glaucon the son of Ariston,
that I might offer up my prayers to the goddess (Bendis, the Thracian
Artemis.); and also because I wanted to see in what manner they would
celebrate the festival, which was a new thing. I was delighted with the
procession of the inhabitants; but that of the Thracians was equally,
if not more, beautiful. When we had finished our prayers and viewed the
spectacle, we turned in the direction of the city; and at that instant
Polemarchus the son of Cephalus chanced to catch sight of us from a
distance as we were starting on our way home, and told his servant to
run and bid us wait for him. The servant took hold of me by the cloak
behind, and said: Polemarchus desires you to wait.

I turned round, and asked him where his master was.

There he is, said the youth, coming after you, if you will only wait.

Certainly we will, said Glaucon; and in a few minutes Polemarchus
appeared, and with him Adeimantus, Glaucon's b

A partir de un rápido vistazo al fragmento de texto anterior podemos ver ciertas cuestiones que tendremos que procesar:
- Las cabeceras de los capítulos.
- Muchos signos de puntuación.
- Nombres extraños.
- Algunos monólogos muy largos. 

### Cargando el texto

El primer paso consiste en cargar el texto en memoria. Podemos desarrollar una pequeña función que se encargue de esto. 

In [1]:
def load_doc(filename):
    # Abrimos el fichero en modo lectura
    file = open(filename,'r')
    # Leemos el texto completo
    text = file.read()
    # Cerramos el fichero
    file.close()
    return text

Usando dicha función podemos cargar nuestro fichero del siguiente modo.

In [4]:
in_filename = 'republic.txt'
doc = load_doc(in_filename)

Ahora podemos mostrar parte de dicho texto.

In [5]:
print(doc[:200])

BOOK I.

I went down yesterday to the Piraeus with Glaucon the son of Ariston,
that I might offer up my prayers to the goddess (Bendis, the Thracian
Artemis.); and also because I wanted to see in what


### Limpiando el texto

Ahora necesitamos transformar el texto en bruto a una secuencia de tokens (o palabras) que podamos usar para entrenar nuestro modelo. 

Vamos a aplicar las siguientes operaciones para limpiar nuestro texto:
- Reemplazar todas las ocurrencias de '-' con un espacio en blanco de manera que podamos partir mejor las palabras.
- Partir las palabras basándonos en espacios en blanco.
- Eliminar todos los símbolos de puntuación.
- Eliminar todas las palabras que no son alfabéticas. 
- Normalizar todas las palabras a minúsculas.

La mayoría de estas transformaciones tienen como objetivo reducir el tamaño del vocabulario. Un tamaño de vocabulario excesivamente grande es un problema cuando se intenta crear modelos de lenguaje. Vocabularios pequeños producen modelos más pequeños que se entrenan más rápidos.

Vamos a implementar todas las operaciones de limpieza en la siguiente función.

In [2]:
import string
import re

def clean_doc(doc):
    # Reemplazar '--' con un espacio en blanco ' '
    doc = doc.replace('--',' ')
    # Partir palabras basándonos en espacios en blanco
    tokens = doc.split()
    # Vamos a escapar las palabras para poder filtrarlas por caracteres
    re_punc = re.compile('[%s]' % re.escape(string.punctuation))
    # Eliminamos los símbolos de puntuación
    tokens = [re_punc.sub('',w) for w in tokens]
    # Eliminamos elementos que nos son alfabéticos
    tokens = [word for word in tokens if word.isalpha()]
    # Convertimos a minúsculas
    tokens = [word.lower() for word in tokens]
    return tokens

Procedemos a limpiar nuestro documento y a continuación mostramos algunas estadísticas sobre nuestro vocabulario.

In [7]:
tokens = clean_doc(doc)
print(tokens[:200])

['book', 'i', 'i', 'went', 'down', 'yesterday', 'to', 'the', 'piraeus', 'with', 'glaucon', 'the', 'son', 'of', 'ariston', 'that', 'i', 'might', 'offer', 'up', 'my', 'prayers', 'to', 'the', 'goddess', 'bendis', 'the', 'thracian', 'artemis', 'and', 'also', 'because', 'i', 'wanted', 'to', 'see', 'in', 'what', 'manner', 'they', 'would', 'celebrate', 'the', 'festival', 'which', 'was', 'a', 'new', 'thing', 'i', 'was', 'delighted', 'with', 'the', 'procession', 'of', 'the', 'inhabitants', 'but', 'that', 'of', 'the', 'thracians', 'was', 'equally', 'if', 'not', 'more', 'beautiful', 'when', 'we', 'had', 'finished', 'our', 'prayers', 'and', 'viewed', 'the', 'spectacle', 'we', 'turned', 'in', 'the', 'direction', 'of', 'the', 'city', 'and', 'at', 'that', 'instant', 'polemarchus', 'the', 'son', 'of', 'cephalus', 'chanced', 'to', 'catch', 'sight', 'of', 'us', 'from', 'a', 'distance', 'as', 'we', 'were', 'starting', 'on', 'our', 'way', 'home', 'and', 'told', 'his', 'servant', 'to', 'run', 'and', 'bid',

In [8]:
print('Total Tokens: %d' % len(tokens))
print('Unique Tokens: %d' %len(set(tokens)))

Total Tokens: 118684
Unique Tokens: 7409


Es decir, nuestro modelo consta de una 7500 palabras. Este tamaño de vocabulario es pequeño y va a ser manejable.

### Guardando el texto limpio

Vamos a organizar la larga lista de tokens en secuencias de 50 palabras de entrada y 1 palabra de salida (esto servirá para luego entrenar nuestro modelo). 

Este proceso lo implementamos con la siguiente función.

In [3]:
def organize_tokens(tokens,input_len=50,output_len=1):
    length = input_len + output_len
    sequences = list()
    for i in range(length,len(tokens)):
        # Elegimos la secuencia de tokens
        seq = tokens[i-length:i]
        # Convertimos la secuencia en una línea
        line = ' '.join(seq)
        # Almacenamos el resultado
        sequences.append(line)
    return sequences

Organizamos nuestros tokens. 

In [10]:
lines = organize_tokens(tokens)

Ahora vamos a guardar las secuencias en un nuevo fichero para poder cargarlo en el futuro. Para ello nos definimos la siguiente función que guardará cada elemento de la secuencia en una línea del fichero. 

In [4]:
def save_doc(lines,filename):
    data = '\n'.join(lines)
    file = open(filename,'w')
    file.write(data)
    file.close()

Podemos llamar a la función anterior para guardar nuestro fichero.

In [12]:
out_filename = 'republic_sequences.txt'
save_doc(lines,out_filename)

Podemos ver parte de dicho fichero.

In [13]:
!head -5 republic_sequences.txt

book i i went down yesterday to the piraeus with glaucon the son of ariston that i might offer up my prayers to the goddess bendis the thracian artemis and also because i wanted to see in what manner they would celebrate the festival which was a new thing i was
i i went down yesterday to the piraeus with glaucon the son of ariston that i might offer up my prayers to the goddess bendis the thracian artemis and also because i wanted to see in what manner they would celebrate the festival which was a new thing i was delighted
i went down yesterday to the piraeus with glaucon the son of ariston that i might offer up my prayers to the goddess bendis the thracian artemis and also because i wanted to see in what manner they would celebrate the festival which was a new thing i was delighted with
went down yesterday to the piraeus with glaucon the son of ariston that i might offer up my prayers to the goddess bendis the thracian artemis and also because i wanted to see in what manner they would

## Entrenando el modelo de lenguaje

Vamosa  entrenar ahora nuestro modelo a partir de los datos que hemos preparado. Dicho modelo tendrá ciertas características:
- Usará una representación para las palabras de manera que palabras diferentes con significados similares tendrán una representación similar.
- La representación será aprendida al mismo tiempo que se aprende el modelo.
- Aprenderá a predecir la probabilidad de la siguiente palabra a partir del contexto de las últimas 100 palabras.

En concreto para implementar este modelo vamos a usar una capa de Embedding para aprender la representación de las palabras, y una red neuronal recurrente con capas LSTM para predecir nuevas palabras basándonos en el contexto. 

### Cargando las secuencias

Podemos comenzar cargando las secuencias que hemos guardado anteriormente. En este caso este paso no sería necesario ya que el proceso de generación de las secuencias es bastante rápido, pero si estamos trabajando con un dataset más grande sí que puede ser conveniente. 


In [14]:
in_filename = 'republic_sequences.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')

### Codificando las secuencias

Las capas de Embedding esperan que las secuencias de entrada estén compuestas de vectores de enteros. Para ello vamos a identificar cada palabra de nuestro vocabulario con un entero único y codificarlo en una secuencia de entrada. En el futuro cuando vayamos a realizar las predicciones tendremos que realizar el proceso inverso.

Para llevar a cabo este proceso de tokenización vamos a usar la API de Keras del siguiente modo. 

In [5]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer

In [16]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)

Ahora podemos acceder a los identificadores de cada palabra usando el atributo ``word_index`` del objeto ``Tokenizer`` que hemos creado. 

Además debemos determinar el tamaño de nuestro vocabulario para definir la capa de embedding. En concreto, a las palabras de nuestro vocabulario se les han asignado valores entre 1 y el número total de palabras de nuestro vocabulario. 

In [17]:
vocab_size = len(tokenizer.word_index) + 1 

### Secuencias de entrada y salida

Una vez que tenemos codificadas nuestras secuencias tenemos que separarlas en elementos de entrada ($X$) y de salida ($y$). Después de realizar la separación debemos codificar cada palabra usando el método one-hot. Este proceso lo llevaremos a cabo mediante la función ``to_categorical()`` de Keras.
Finalmente necesitamos especificar cómo de largas serán las secuencias de entrada. 

In [6]:
from tensorflow.keras.utils import to_categorical
from numpy import array 

In [18]:
sequences = array(sequences)
X,y=sequences[:,:-1], sequences[:,-1]
y = to_categorical(y,num_classes=vocab_size)
seq_length = X.shape[1]

### Entrenando el modelo

Ahora podemos definir nuestro modelo que constará de una capa de Embedding, seguida de dos capas LSTM y terminando con una red completamente conectada. 

In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.optimizers import Adam

In [8]:
def define_model(vocab_size,seq_length):
    model = Sequential()
    model.add(Embedding(vocab_size,50,input_length=seq_length))
    model.add(LSTM(100,return_sequences=True))
    model.add(LSTM(100))
    model.add(Dense(100,activation='relu'))
    model.add(Dense(vocab_size,activation='softmax'))
    model.compile(loss='categorical_crossentropy',optimizer=Adam(),metrics=['accuracy'])
    return model

Pasamos a entrenar nuestro modelo. Como este proceso es bastante costoso (incluso usando GPUs) en la siguiente sección se proporcionan los ficheros necesarios para usar el modelo. 

In [20]:
model = define_model(vocab_size,seq_length)
model.fit(X,y,batch_size=128,epochs=100)

Epoch 1/100
927/927 [==============================] - 47s 17ms/step - loss: 6.4985 - accuracy: 0.0576
Epoch 2/100
927/927 [==============================] - 15s 17ms/step - loss: 5.7338 - accuracy: 0.1000
Epoch 3/100
927/927 [==============================] - 15s 17ms/step - loss: 5.4764 - accuracy: 0.1274
Epoch 4/100
927/927 [==============================] - 15s 17ms/step - loss: 5.2920 - accuracy: 0.1406
Epoch 5/100
927/927 [==============================] - 16s 17ms/step - loss: 5.1544 - accuracy: 0.1550
Epoch 6/100
927/927 [==============================] - 16s 17ms/step - loss: 5.0618 - accuracy: 0.1592
Epoch 7/100
927/927 [==============================] - 16s 17ms/step - loss: 5.0797 - accuracy: 0.1545
Epoch 8/100
927/927 [==============================] - 16s 17ms/step - loss: 4.9621 - accuracy: 0.1674
Epoch 9/100
927/927 [==============================] - 16s 17ms/step - loss: 4.8789 - accuracy: 0.1704
Epoch 10/100
927/927 [==============================] - 16s 17ms/step - l

Una vez entrenado podemos guardar los pesos del modelo y el tokenizador. 

In [21]:
model.save_weights('./model.h5', overwrite=True)

In [9]:
from pickle import dump

In [22]:
dump(tokenizer, open('tokenizer.pkl','wb'))

## Usando el modelo

Como has podido ver en el paso anterior, el proceso de entrenar este tipo de modelos es muy costoso, por lo que puedes descargar los ficheros necesarios para usar el modelo desde el siguiente enlace. 

In [23]:
!wget https://raw.githubusercontent.com/ts1819/datasets/master/practica5/tpu_model.h5 -O model.h5
!wget https://raw.githubusercontent.com/ts1819/datasets/master/practica5/tokenizer.pkl -O tokenizer.pkl

--2021-05-15 09:07:32--  https://raw.githubusercontent.com/ts1819/datasets/master/practica5/tpu_model.h5
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5101248 (4.9M) [application/octet-stream]
Saving to: ‘model.h5’

model.h5            100%[===================>]   4.86M  29.4MB/s    in 0.2s    

2021-05-15 09:07:33 (29.4 MB/s) - ‘model.h5’ saved [5101248/5101248]

--2021-05-15 09:07:33--  https://raw.githubusercontent.com/ts1819/datasets/master/practica5/tokenizer.pkl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 353379 (345K)

### Cargando los datos

Comenzamos cargando nuestros datos al igual que antes. 

In [24]:
in_filename = 'republic_sequences.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')

Necesitamos este texto para elegir una secuencia de inicio que será la entrada para nuestro modelo. 

In [25]:
seq_length = len(lines[0].split())-1

### Cargando el modelo

Vamos a cargar el modelo y a fijar los pesos. Notar que para este paso ya no necesitamos el uso de TPU, y que el modelo podría ser usado en cualquier ordenador.

In [26]:
model = define_model(vocab_size,seq_length)
model.load_weights('./model.h5')

También necesitamos cargar el tokenizador.

In [10]:
from pickle import load

In [27]:
tokenizer = load(open('tokenizer.pkl','rb'))

### Generando texto

El primer paso para generar el texto consiste en preparar una entrada, para lo cual elegiremos una línea aleatoria del texto. 

In [11]:
from random import randint

In [28]:
seed_text = lines[randint(0,len(lines))]
print(seed_text + '\n')

great utility of having wives and children in common the possibility is quite another matter and will be very much disputed i think that a good many doubts may be raised about both you imply that the two questions must be combined i replied now i meant that you should admit



A continuación podemos generar nuevas palabras una por una. Primero, el texto debe codificarse usando el tokenizer que hemos cargado anteriormente. Ahora el modelo puede predecir nuevas palabras usando el método ``predict_classes()`` que devuelve el índice de la palabra con probabilidad más alta. 

Esta palabra se añade a nuestro texto inicial y se repite el proceso. Notar que esta secuencia va a ir creciendo por lo que tendremos que truncarla, para lo que utilizamos la función ``pad_sequences()`` de Keras. Todo este proceso se puede implementar con la siguiente función. 

In [12]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

def generate_seq(model,tokenizer,seq_length,seed_text,n_words):
    result = list()
    in_text = seed_text
    for _ in range(n_words):
        encoded = tokenizer.texts_to_sequences([in_text])[0]
        encoded = pad_sequences([encoded],maxlen=seq_length,truncating='pre')
        yhat = model.predict_classes(encoded,verbose=0)
        out_word = ''
        for word,index in tokenizer.word_index.items():
            if index == yhat:
                out_word = word
                break
        in_text += ' ' + out_word
        result.append(out_word)
    return ' '.join(result)

Ahora podemos generar una nueva secuencia usando el siguiente código. Cada vez que lo ejecutemos obtendremos un resultado distinto.

In [30]:
seed_text = lines[randint(0,len(lines))]
print(seed_text + '\n')
generated = generate_seq(model,tokenizer,seq_length,seed_text,50)
print(generated)

of payment is medicine i should not nor would you say that medicine is the art of receiving pay because a man takes fees when he is engaged in healing certainly not and we have admitted i said that the good of each art is specially confined to the art yes



/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


he said the same principle holds of the other part of the other part of the other part of the other part of the other part of the other part of the other part of the other part of the other degree that resides of the state constitutes courage in


## Ejercicio

Elige tu propio libro del proyecto Gutenberg (es posible usar libros en [español](https://www.gutenberg.org/browse/languages/es)) y crea tu propio modelo de lenguaje. 

In [13]:
!wget https://www.dropbox.com/s/g9v964y5y2yl94j/regenta.txt?dl=1 -O regenta.txt

--2021-05-15 09:57:35--  https://www.dropbox.com/s/g9v964y5y2yl94j/regenta.txt?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.18, 2620:100:601a:18::a27d:712
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/dl/g9v964y5y2yl94j/regenta.txt [following]
--2021-05-15 09:57:35--  https://www.dropbox.com/s/dl/g9v964y5y2yl94j/regenta.txt
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucd9761b1d63ff3385cbb39b5dc7.dl.dropboxusercontent.com/cd/0/get/BOgdz9XNbccMnwD3k3RQcUWXs7zTzFsu2pwmRP0wlPFe-GhFBMTe_4UokELYiA_Zsv4veeJPiUcFJPFaOTobIi0ExA27ooRlewkr4jQ_uKxGZkcld9l3I8daCJ_BE_ACn97QfnLsA1Pn2HC_EinhWcg6/file?dl=1# [following]
--2021-05-15 09:57:35--  https://ucd9761b1d63ff3385cbb39b5dc7.dl.dropboxusercontent.com/cd/0/get/BOgdz9XNbccMnwD3k3RQcUWXs7zTzFsu2pwmRP0wlPFe-GhFBMTe_4UokELYiA_Zsv4veeJPiUcFJPFaOTobIi0ExA2

**Preparando los datos**

In [14]:
!head -30 regenta.txt

﻿Prólogo


Creo que fue Wieland quien dijo _que los pensamientos de los hombres
valen más que sus acciones, y las buenas novelas más que el género
humano_. Podrá esto no ser verdad; pero es hermoso y consolador.
Ciertamente, parece que nos ennoblecemos trasladándonos de este mundo al
otro, de la realidad en que somos tan malos a la ficción en que valemos
más que aquí, y véase por qué, cuando un cristiano el hábito de pasar
fácilmente a mejor vida, inventando personas y tejiendo sucesos a imagen
de los de por acá, le cuesta no poco trabajo volver a este mundo.
También digo que si grata es la tarea de fabricar género humano
recreándonos en ver cuánto superan las ideales figurillas, por toscas
que sean, a las vivas figuronas que a nuestro lado bullen, el regocijo
es más intenso cuando visitamos los talleres ajenos, pues el andar
siempre en los propios trae un desasosiego que amengua los placeres de
lo que llamaremos creación, por no tener mejor nombre que darle.

Esto que digo de visitar 

**Cargando los datos**

In [15]:
in_filename = 'regenta.txt'
doc = load_doc(in_filename)

In [16]:
print(doc[:200])

﻿Prólogo


Creo que fue Wieland quien dijo _que los pensamientos de los hombres
valen más que sus acciones, y las buenas novelas más que el género
humano_. Podrá esto no ser verdad; pero es hermoso y 


**Limpiando los datos**

In [17]:
tokens = clean_doc(doc)
print(tokens[:200])

['creo', 'que', 'fue', 'wieland', 'quien', 'dijo', 'que', 'los', 'pensamientos', 'de', 'los', 'hombres', 'valen', 'más', 'que', 'sus', 'acciones', 'y', 'las', 'buenas', 'novelas', 'más', 'que', 'el', 'género', 'humano', 'podrá', 'esto', 'no', 'ser', 'verdad', 'pero', 'es', 'hermoso', 'y', 'consolador', 'ciertamente', 'parece', 'que', 'nos', 'ennoblecemos', 'trasladándonos', 'de', 'este', 'mundo', 'al', 'otro', 'de', 'la', 'realidad', 'en', 'que', 'somos', 'tan', 'malos', 'a', 'la', 'ficción', 'en', 'que', 'valemos', 'más', 'que', 'aquí', 'y', 'véase', 'por', 'qué', 'cuando', 'un', 'cristiano', 'el', 'hábito', 'de', 'pasar', 'fácilmente', 'a', 'mejor', 'vida', 'inventando', 'personas', 'y', 'tejiendo', 'sucesos', 'a', 'imagen', 'de', 'los', 'de', 'por', 'acá', 'le', 'cuesta', 'no', 'poco', 'trabajo', 'volver', 'a', 'este', 'mundo', 'también', 'digo', 'que', 'si', 'grata', 'es', 'la', 'tarea', 'de', 'fabricar', 'género', 'humano', 'recreándonos', 'en', 'ver', 'cuánto', 'superan', 'las', 

In [18]:
print('Total Tokens: %d' % len(tokens))
print('Unique Tokens: %d' %len(set(tokens)))

Total Tokens: 131766
Unique Tokens: 15467


Es decir, nuestro modelo consta de unas 15500 palabras. Este tamaño de vocabulario es manejable (recortando un tercio de la obra original por una limitación de memoria RAM). 

**Guardando los datos limpios**

In [19]:
lines = organize_tokens(tokens)

In [20]:
out_filename = 'regenta_sequences.txt'
save_doc(lines, out_filename)

In [21]:
!head -5 regenta_sequences.txt

creo que fue wieland quien dijo que los pensamientos de los hombres valen más que sus acciones y las buenas novelas más que el género humano podrá esto no ser verdad pero es hermoso y consolador ciertamente parece que nos ennoblecemos trasladándonos de este mundo al otro de la realidad en
que fue wieland quien dijo que los pensamientos de los hombres valen más que sus acciones y las buenas novelas más que el género humano podrá esto no ser verdad pero es hermoso y consolador ciertamente parece que nos ennoblecemos trasladándonos de este mundo al otro de la realidad en que
fue wieland quien dijo que los pensamientos de los hombres valen más que sus acciones y las buenas novelas más que el género humano podrá esto no ser verdad pero es hermoso y consolador ciertamente parece que nos ennoblecemos trasladándonos de este mundo al otro de la realidad en que somos
wieland quien dijo que los pensamientos de los hombres valen más que sus acciones y las buenas novelas más que el género humano po

**Entrenando el modelo**

In [22]:
in_filename = 'regenta_sequences.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')

In [23]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)

In [24]:
vocab_size = len(tokenizer.word_index) + 1

In [25]:
sequences = array(sequences)
X, y = sequences[:, :-1], sequences[:, -1]
y = to_categorical(y, num_classes=vocab_size)
seq_length = X.shape[1]

In [26]:
model = define_model(vocab_size, seq_length)
model.fit(X, y, batch_size=128, epochs=100)

Epoch 1/100
1030/1030 [==============================] - 57s 25ms/step - loss: 7.3158 - accuracy: 0.0514
Epoch 2/100
1030/1030 [==============================] - 25s 25ms/step - loss: 6.5169 - accuracy: 0.0751
Epoch 3/100
1030/1030 [==============================] - 25s 25ms/step - loss: 6.2779 - accuracy: 0.0837
Epoch 4/100
1030/1030 [==============================] - 25s 25ms/step - loss: 6.0902 - accuracy: 0.0968
Epoch 5/100
1030/1030 [==============================] - 25s 25ms/step - loss: 5.9411 - accuracy: 0.1028
Epoch 6/100
1030/1030 [==============================] - 26s 25ms/step - loss: 5.7954 - accuracy: 0.1071
Epoch 7/100
1030/1030 [==============================] - 26s 25ms/step - loss: 5.6472 - accuracy: 0.1109
Epoch 8/100
1030/1030 [==============================] - 25s 25ms/step - loss: 5.5158 - accuracy: 0.1163
Epoch 9/100
1030/1030 [==============================] - 26s 25ms/step - loss: 5.4175 - accuracy: 0.1221
Epoch 10/100
1030/1030 [==============================]

In [27]:
model.save_weights('./model2.h5', overwrite=True)

In [28]:
dump(tokenizer, open('tokenizer2.pkl','wb'))

**Usando el modelo**

In [29]:
seq_length = len(lines[0].split()) - 1

In [30]:
seed_text = lines[randint(0, len(lines))]
print(seed_text + '\n')

salgo estas palabras apenas dichas le parecieron imprudentes ella quien las había pronunciado así hablaba obdulia con los hombres ella ana don álvaro se vio en un apuro pretendía aquella señora una conversación para aludir a lo que había entre ellos que en rigor no era nada que mereciese comentarios él



In [31]:
seed_text = lines[randint(0, len(lines))]
print(seed_text + '\n')
generated = generate_seq(model, tokenizer, seq_length, seed_text, 50)
print(generated)

jugaban al tute la presencia del provisor interrumpió el juego los familiares se pusieron de pie y uno de ellos hermoso rubio de movimientos suaves y ondulantes de pulquérrimo traje talar perfumado abrió una mampara forrada de damasco color cereza de lo mismo estaba tapizada toda la estancia que se vio



/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


a solas y se le antojaba a la marquesa estando decidida bien trepando poco a mí y se le antojaba de tus precauciones predominaba un castigo de la estación y con poder que para cazar gorriones no es la muerte de la catedral y la regenta se le había figurado


Recuerda guardar este notebook en tu repositorio usando la opción "Save in GitHub" del menú File.